# Pair RDDs

- Definition: Key-value pairs, commonly for operations such as aggregations and ETL
    + e.g. student ID as key and their information as values
    + Key: int, str, tuples, etc. 
    + Values: lists, tuples, dicts, set, etc. - can be simple obj to data structures
- Key could be duplicate, same key can appear in each element
- Allow operations on each key in parallel . regroup data cross the network

### Example 1

In [1]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [2]:
lines = sc.textFile("./data/README.md")
words = lines.flatMap(lambda l: l.split(" ")).map(lambda x: (x, 1))

In [3]:
words.glom().collect()[:10]

[[(u'#', 1),
  (u'Apache', 1),
  (u'Spark', 1),
  (u'', 1),
  (u'Spark', 1),
  (u'is', 1),
  (u'a', 1),
  (u'fast', 1),
  (u'and', 1),
  (u'general', 1),
  (u'cluster', 1),
  (u'computing', 1),
  (u'system', 1),
  (u'for', 1),
  (u'Big', 1),
  (u'Data.', 1),
  (u'It', 1),
  (u'provides', 1),
  (u'high-level', 1),
  (u'APIs', 1),
  (u'in', 1),
  (u'Scala,', 1),
  (u'Java,', 1),
  (u'Python,', 1),
  (u'and', 1),
  (u'R,', 1),
  (u'and', 1),
  (u'an', 1),
  (u'optimized', 1),
  (u'engine', 1),
  (u'that', 1),
  (u'supports', 1),
  (u'general', 1),
  (u'computation', 1),
  (u'graphs', 1),
  (u'for', 1),
  (u'data', 1),
  (u'analysis.', 1),
  (u'It', 1),
  (u'also', 1),
  (u'supports', 1),
  (u'a', 1),
  (u'rich', 1),
  (u'set', 1),
  (u'of', 1),
  (u'higher-level', 1),
  (u'tools', 1),
  (u'including', 1),
  (u'Spark', 1),
  (u'SQL', 1),
  (u'for', 1),
  (u'SQL', 1),
  (u'and', 1),
  (u'DataFrames,', 1),
  (u'MLlib', 1),
  (u'for', 1),
  (u'machine', 1),
  (u'learning,', 1),
  (u'GraphX', 

## Pair RDDs Operations

### Transformation

- **`keys()`**: return an RDD of keys only (allow duplicates)
- **`values()`**: return an RDD of values only (allow duplications)


- **`sortByKey()`**: return an RDD sorted by the key - option to set ascending=True/False
- **`groupByKey()`**: group value of the same key: `rdd.groupByKey().map(lambda x: (x[0], list(x[1]))` to visualize


- **`mapValues()`**: pass each value in the key-value pair RDD through a map fucntion without changing the keys. Retain the original partioning
- **`flatMapValues()`**: pass each value in the key-value pair RDD through a flatMap function without changing the keys. Retain the original RDD's partitioning. 


- **`reduceByKey()`**: Run several parallel reduce operations, one for each key in the data set. When called on a dataset of (Key, Val) pairs. Returns a dataset of (Key, Val) pairs where the values for each key are aggregated using the given reduce function func.
    + calculate the function (value) on the partition first, and then merge using the unique key
    + reduceByKey and groupByKey will require less data shuffle by grouping the values together by key vs. mapValues (i.e. more computationally expensive)
    + To group values for the purpose of aggregation using reduceByKey(), foldByKey() or combineByKey() will provide worse performance since they combines the aggregation function before the shuffle.
    
    
- **`combineByKey(createCombiner, mergeValue, mergeCombiners)`**:
    + Similar to `aggregate()`.
    + **`createCombiner`** - If it is new in a partition (not an RDD!), create the initial value for the accumulator on the key.
    + **`mergeValue`** - If it is not new in the partition, apply the mergeValue function. 
    + **`mergeCombiners`** - Since each partition is processed independently, we can have multiple accumulators for the same key. When merging the results from each partition, apply the mergeCombiners to merge the accumulators for the same key.

- Operations don't shuffle data: `mapValues`, `flatMapValues`. 
- Operations shuffle data: `groupByKey`, `reduceByKey`, `combineByKey`, `sortByKey`

### Example 2

In [4]:
# generate key-value pairs of (length of word, word)
words = lines.flatMap(lambda l: l.split(" "))
words_length = words.map(lambda x: (len(x),x))
words_length.collect()[:10]

[(1, u'#'),
 (6, u'Apache'),
 (5, u'Spark'),
 (0, u''),
 (5, u'Spark'),
 (2, u'is'),
 (1, u'a'),
 (4, u'fast'),
 (3, u'and'),
 (7, u'general')]

In [5]:
words_length.keys().collect()[:10]

[1, 6, 5, 0, 5, 2, 1, 4, 3, 7]

In [6]:
words_length.values().collect()[:10]

[u'#',
 u'Apache',
 u'Spark',
 u'',
 u'Spark',
 u'is',
 u'a',
 u'fast',
 u'and',
 u'general']

In [7]:
words_length.sortByKey(ascending=True).distinct().collect()[:10]

[(3, u'one'),
 (6, u'stream'),
 (4, u'Hive'),
 (7, u'package'),
 (7, u'module,'),
 (5, u'them,'),
 (4, u'uses'),
 (3, u'set'),
 (6, u'thread'),
 (5, u'start')]

### Example 3

In [8]:
# create a pair RDD with (length of a word, list of words) from “README.md”.
# return iterable object
words_length.groupByKey().collect()[:10]

[(0, <pyspark.resultiterable.ResultIterable at 0x10ba33d10>),
 (112, <pyspark.resultiterable.ResultIterable at 0x10ba3f1d0>),
 (2, <pyspark.resultiterable.ResultIterable at 0x10ba3f250>),
 (4, <pyspark.resultiterable.ResultIterable at 0x10ba3f290>),
 (6, <pyspark.resultiterable.ResultIterable at 0x10ba3f2d0>),
 (8, <pyspark.resultiterable.ResultIterable at 0x10ba3f310>),
 (96, <pyspark.resultiterable.ResultIterable at 0x10ba3f350>),
 (10, <pyspark.resultiterable.ResultIterable at 0x10ba3f390>),
 (12, <pyspark.resultiterable.ResultIterable at 0x10ba3f3d0>),
 (18, <pyspark.resultiterable.ResultIterable at 0x10ba3f410>)]

In [9]:
# return words with length=10
words_length.groupByKey().map(lambda x: (x[0], list(x[1]))).filter(lambda x: x[0]==10).collect()

[(10,
  [u'high-level',
   u'downloaded',
   u'["Parallel',
   u'["Building',
   u'developing',
   u'`examples`',
   u'directory.',
   u'[params]`.',
   u'"local[N]"',
   u'`examples`',
   u'individual',
   u'particular',
   u'particular'])]

### Example 4

In [10]:
# generate key-value pairs of (word, occurence)
words = lines.flatMap(lambda l: l.split(" "))
words_occr = words.map(lambda x: (x,1)).groupByKey().map(lambda x: (x[0], list(x[1])))

In [11]:
words_occr = words_occr.map(lambda x: (x[0], sum(x[1])))
words_occr.collect()[:10]

[(u'', 68),
 (u'when', 1),
 (u'R,', 1),
 (u'including', 3),
 (u'computation', 1),
 (u'using:', 1),
 (u'guidance', 2),
 (u'Scala,', 1),
 (u'environment', 1),
 (u'only', 1)]

### Example 5

In [10]:
word = sc.textFile("./data/README.md").filter(lambda x : "spark" in x)

In [11]:
len_word_pair = word.map(lambda x : (len(x),x))
len_word_pair_group = len_word_pair.groupByKey()

In [12]:
len_word_pair_group.mapValues(lambda x : x).glom().collect()[:5]

[[(76, <pyspark.resultiterable.ResultIterable at 0x10f6a6850>),
  (84, <pyspark.resultiterable.ResultIterable at 0x10f6a6bd0>),
  (54, <pyspark.resultiterable.ResultIterable at 0x10f6a67d0>),
  (120, <pyspark.resultiterable.ResultIterable at 0x10f6a65d0>),
  (26, <pyspark.resultiterable.ResultIterable at 0x10f6a6790>),
  (62, <pyspark.resultiterable.ResultIterable at 0x10f6a6650>)],
 [(97, <pyspark.resultiterable.ResultIterable at 0x10f784510>),
  (21, <pyspark.resultiterable.ResultIterable at 0x10f784050>),
  (17, <pyspark.resultiterable.ResultIterable at 0x10f784190>)]]

In [17]:
len_word_pair_group.flatMapValues(lambda x : x).glom().collect()[:5]

[[(76,
   u'guide, on the [project web page](http://spark.apache.org/documentation.html)'),
  (76,
   u'["Building Spark"](http://spark.apache.org/docs/latest/building-spark.html).'),
  (84,
   u'Testing first requires [building Spark](#building-spark). Once Spark is built, tests'),
  (54, u'    MASTER=spark://host:7077 ./bin/run-example SparkPi'),
  (120,
   u'["Specifying the Hadoop Version"](http://spark.apache.org/docs/latest/building-spark.html#specifying-the-hadoop-version)'),
  (26, u'<http://spark.apache.org/>'),
  (62, u'examples to a cluster. This can be a mesos:// or spark:// URL,')],
 [(97,
   u'Please refer to the [Configuration Guide](http://spark.apache.org/docs/latest/configuration.html)'),
  (21, u'    ./bin/spark-shell'),
  (17, u'    ./bin/pyspark')]]

### Example 6

In [12]:
# generate key-value pairs of (word, occurence) using reduceByKey
words = lines.flatMap(lambda l: l.split(" "))
words_occr = words.map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)

In [13]:
words_occr.collect()[:10]

[(u'', 68),
 (u'when', 1),
 (u'R,', 1),
 (u'including', 3),
 (u'computation', 1),
 (u'using:', 1),
 (u'guidance', 2),
 (u'Scala,', 1),
 (u'environment', 1),
 (u'only', 1)]

### Example 7

In [14]:
# create pairs (length of words, (frequency, a list of words)) using combineByKey()
# step: textFile -> flatMap(x.split()) -> map((len(x), x))
words_length = words.map(lambda x: (len(x),x))
words_length.collect()[:10]

[(1, u'#'),
 (6, u'Apache'),
 (5, u'Spark'),
 (0, u''),
 (5, u'Spark'),
 (2, u'is'),
 (1, u'a'),
 (4, u'fast'),
 (3, u'and'),
 (7, u'general')]

In [15]:
word_pairs = words_length.combineByKey((lambda x: (1, x)), #createCombiner - if you see the word for the first time
                                      (lambda x, y: (x[0]+1, x[1]+","+y)), # mergeValue
                                      (lambda x, y: (x[0]+y[0], x[1]+","+y[1]))) # mergeCombiner

word_pairs.sortBy(lambda x: x[0]).collect()[:5]                   

[(0,
  (68,
   u',,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,')),
 (1, (11, u'#,a,a,a,a,a,a,N,a,A,a')),
 (2,
  (70,
   u'is,It,in,R,,an,It,of,##,on,##,is,To,do,to,do,if,by,-T,in,is,at,an,##,to,is,to,##,if,##,in,To,of,Pi,to,to,be,or,to,on,to,or,to,an,if,is,in,of,if,no,##,is,be,on,to,or,##,to,to,in,of,to,at,on,of,##,to,in,an,on,to')),
 (3,
  (94,
   u'and,for,Big,and,and,for,set,SQL,for,SQL,and,for,for,and,for,You,can,the,the,web,and,and,its,not,you,You,can,one,the,see,the,For,see,and,The,way,the,Try,the,you,you,can,use,the,And,run,the,>>>,the,run,one,use,For,run,the,You,can,set,the,can,run,and,run,one,run,You,can,use,the,the,For,the,are,can,run,see,the,how,for,the,and,the,you,the,the,the,for,for,for,and,the,the,for,how')),
 (4,
  (47,
   u'fast,APIs,that,data,also,rich,find,This,file,only,run:,(You,need,this,more,than,with,More,from,IDE,,also,also,with,will,when,This,URL,,with,with,also,name,Many,help,Once,[run,Note,uses,core,talk,HDFS,have,must,same,that,your,Hiv